In [2]:
import pandas as pd
data=pd.read_csv('../data/partidos_limpio.csv')

In [3]:
# Extracción de goles de local y visitante de la columna 'Score'
data['Home Goals'] = data['Score'].apply(lambda x: int(x.split('-')[0].strip()))
data['Away Goals'] = data['Score'].apply(lambda x: int(x.split('-')[1].strip()))

# Creación de una columna para el resultado del partido
# 1 si gana el local, -1 si gana el visitante, 0 si empate
data['Match Result'] = data.apply(lambda x: 1 if x['Home Goals'] > x['Away Goals'] else (-1 if x['Home Goals'] < x['Away Goals'] else 0), axis=1)

# Vamos a mostrar los cambios y las nuevas columnas
data[['Home', 'Away', 'Score', 'Home Goals', 'Away Goals', 'Match Result']].head()


,Home,Away,Score,Home Goals,Away Goals,Match Result
0,Milan,Tottenham,1-0,1,0,1
1,Paris S-G,Bayern Munich,0-1,0,1,-1
2,Club Brugge,Benfica,0-2,0,2,-1
3,Dortmund,Chelsea,1-0,1,0,1
4,Liverpool,Real Madrid,2-5,2,5,-1


In [4]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Crear instancias de LabelEncoder
encoder = LabelEncoder()

# Codificar variables categóricas
data['Season_encoded'] = encoder.fit_transform(data['Season'])
data['Round_encoded'] = encoder.fit_transform(data['Round'])
data['Home_encoded'] = encoder.fit_transform(data['Home'])
data['Away_encoded'] = encoder.fit_transform(data['Away'])
data['Venue_encoded'] = encoder.fit_transform(data['Venue'])
data['Referee_encoded'] = encoder.fit_transform(data['Referee'])

# Crear y escalar características numéricas
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data[['Home Goals', 'Away Goals', 'Season_encoded', 'Round_encoded', 'Home_encoded', 'Away_encoded', 'Venue_encoded', 'Referee_encoded']])

# Preparar los datos para el entrenamiento
X = scaled_features
y = data['Match Result'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((459, 8), (115, 8), (459,), (115,))

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Construir el modelo de red neuronal
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')  # Utilizamos activación lineal para una clasificación simple
])

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=10, validation_split=0.2, verbose=0)

# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

test_loss, test_accuracy


(0.014033171348273754, 0.643478274345398)